In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [23]:
import pandas as pd
# Make display smaller
# pd.options.display.max_rows = 10
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
user_set = pd.read_csv('users.dat', sep='::', encoding="utf-8",
 header=None, names=unames, engine='python')
rnames = ['user_id', 'item_id', 'rating', 'timestamp']
rdf = pd.read_csv('ratings.dat', sep='::', encoding="utf-8",
 header=None, names=rnames, engine='python')
mnames = ['item_id', 'title', 'genres']
item_df = pd.read_csv('movies.dat', sep='::', encoding='latin-1',
 header=None, names=mnames, engine='python')

In [24]:
item_genre_dict = dict()
for i in range(len(item_df)):
    genre_str = item_df.at[i, 'genres']
    genre_list = genre_str.split('|')
    item_genre_dict[item_df.at[i, 'item_id']] = genre_list

In [25]:
item_genre_dict

{1: ['Animation', "Children's", 'Comedy'],
 2: ['Adventure', "Children's", 'Fantasy'],
 3: ['Comedy', 'Romance'],
 4: ['Comedy', 'Drama'],
 5: ['Comedy'],
 6: ['Action', 'Crime', 'Thriller'],
 7: ['Comedy', 'Romance'],
 8: ['Adventure', "Children's"],
 9: ['Action'],
 10: ['Action', 'Adventure', 'Thriller'],
 11: ['Comedy', 'Drama', 'Romance'],
 12: ['Comedy', 'Horror'],
 13: ['Animation', "Children's"],
 14: ['Drama'],
 15: ['Action', 'Adventure', 'Romance'],
 16: ['Drama', 'Thriller'],
 17: ['Drama', 'Romance'],
 18: ['Thriller'],
 19: ['Comedy'],
 20: ['Action'],
 21: ['Action', 'Comedy', 'Drama'],
 22: ['Crime', 'Drama', 'Thriller'],
 23: ['Thriller'],
 24: ['Drama', 'Sci-Fi'],
 25: ['Drama', 'Romance'],
 26: ['Drama'],
 27: ['Drama'],
 28: ['Romance'],
 29: ['Adventure', 'Sci-Fi'],
 30: ['Drama'],
 31: ['Drama'],
 32: ['Drama', 'Sci-Fi'],
 33: ['Adventure', 'Romance'],
 34: ["Children's", 'Comedy', 'Drama'],
 35: ['Drama', 'Romance'],
 36: ['Drama'],
 37: ['Documentary'],
 38: ['C

In [26]:
item_set = set(rdf['item_id'].unique())
user_set = set(rdf['user_id'].unique())
print('item num = ' + str(len(item_set)))
print('user num = ' + str(len(user_set)))

item num = 3706
user num = 6040


In [27]:
# count the number for each genre and sort
import operator
genre_count = dict()
for l in item_genre_dict:
    for g in item_genre_dict[l]:
        if not g in genre_count:
            genre_count[g] = 1
        else:
            genre_count[g] += 1

genre_count_sorted = sorted(genre_count.items(), key=operator.itemgetter(1), reverse=True)
genre_count_sorted

[('Drama', 1603),
 ('Comedy', 1200),
 ('Action', 503),
 ('Thriller', 492),
 ('Romance', 471),
 ('Horror', 343),
 ('Adventure', 283),
 ('Sci-Fi', 276),
 ("Children's", 251),
 ('Crime', 211),
 ('War', 143),
 ('Documentary', 127),
 ('Musical', 114),
 ('Mystery', 106),
 ('Animation', 105),
 ('Fantasy', 68),
 ('Western', 68),
 ('Film-Noir', 44)]

In [28]:
key_genre_rating = dict()

for r in range(len(rdf)):
    item = rdf.at[r, 'item_id']
    gl = item_genre_dict[item]
    for k in key_genre:
        if k in gl:
            key_genre_rating[k] += 1.0

NameError: name 'key_genre' is not defined

In [62]:
avg_rating_by_movie_genres = pd.merge(ratings, users, how='left', on='user_id')
avg_rating_by_movie_genres.drop(['occupation', 'zip'], axis=1, inplace=True)
avg_rating_by_movie_genres.head()

,user_id,movie_id,rating,timestamp,gender,age
0,1,1193,5,978300760,F,1
1,1,661,3,978302109,F,1
2,1,914,3,978301968,F,1
3,1,3408,4,978300275,F,1
4,1,2355,5,978824291,F,1


In [87]:
# remove movies that has less than 20 ratings
ratings_per_movie = ratings.groupby(['movie_id']).size().reset_index(name='counts').sort_values(by=['counts'])
ratings_per_movie = ratings_per_movie[ratings_per_movie['counts'] >= 20]
# ratings_per_movie

In [116]:
avg_rating_by_movie_genres = avg_rating_by_movie_genres[avg_rating_by_movie_genres['movie_id'].isin(ratings_per_movie['movie_id'])]
avg_rating_by_movie_genres['user_id'] = avg_rating_by_movie_genres['user_id']-1

In [117]:
#split train 80, tune 10, test 10
train,test_tmp = train_test_split(avg_rating_by_movie_genres, test_size=0.2, random_state=181)

In [118]:
tune,test = train_test_split(test_tmp, test_size=0.5, random_state=181)

In [121]:
train.to_csv('train_df.csv', header=False)
test.to_csv('test_df.csv', header=False)
tune.to_csv('tune_df.csv', header=False)

In [120]:
tune

,user_id,movie_id,rating,timestamp,gender,age
861349,5181,2520,2,961787817,F,35
249126,1501,586,4,975141645,M,18
346631,2030,3852,4,1042347160,M,35
710998,4265,2581,4,965304651,M,56
794678,4753,2405,3,963185357,F,18
...,...,...,...,...,...,...
43347,296,2948,4,976513035,M,18
964775,5813,1223,5,957915763,M,25
828536,4978,923,5,962604769,M,35
542210,3335,1179,2,967962952,M,35


In [10]:
# avg_rating_by_movie = avg_rating_by_movie_genres.groupby(['movie_id','genres'], as_index=False).mean()
# avg_rating_by_movie = avg_rating_by_movie[['movie_id', 'genres', 'rating']]
# avg_rating_by_movie.head()

,movie_id,genres,rating
0,1,Animation|Children's|Comedy,4.146846
1,2,Adventure|Children's|Fantasy,3.201141
2,3,Comedy|Romance,3.016736
3,4,Comedy|Drama,2.729412
4,5,Comedy,3.006757


In [8]:
# avg_rating_by_movie_genres=avg_rating_by_movie_genres[avg_rating_by_movie_genres['genres'].isin(['Drama', 'Horror', 'Comedy', 'Thriller', 'Documentary', 'Action'])]

# len(avg_rating_by_movie_genres)

287296

In [21]:
# avg_rating_by_movie.to_csv('test_adversary2.dat')
# avg_rating_by_movie_genres['genres'] = avg_rating_by_movie_genres['genres'].str.split('|', 1).str[0]

In [23]:
# avg_rating_by_movie_genres.groupby(['genres']).size().reset_index(name='counts').sort_values(by=['counts']).tail(20)

,genres,counts
8,Fantasy,790
16,War,991
13,Romance,2832
17,Western,5689
6,Documentary,6817
11,Musical,7112
9,Film-Noir,9343
12,Mystery,10237
14,Sci-Fi,11464
15,Thriller,17851


In [34]:
item_genre_dict = dict()
for i in range(len(avg_rating_by_movie)):
    genre_str = avg_rating_by_movie.at[i, 'genres']
    genre_list = genre_str.split('|')
    item_genre_dict[avg_rating_by_movie.at[i, 'movie_id']] = genre_list

In [36]:
key_genre = ['Sci-Fi', 'Horror', 'Crime', 'Adventure', "Children's", 'Romance']

# get the key_genre->item_list dict
key_genre_item = dict()
for k in key_genre:
    key_genre_item[k] = list()
for item in item_genre_dict:
    for g in item_genre_dict[item]:
        if g in key_genre:
            key_genre_item[g].append(item)

In [38]:
# collect all the items with key genres
key_item_set = set()
for genre in key_genre_item:
    key_item_set |= set(key_genre_item[genre])

nonkey_item_set = item_set - key_item_set

NameError: name 'item_set' is not defined